# Project: Bangla FAQ Bot using RAG

**Objectives:**

- Selecting 5 Bangla topics (e.g., শিক্ষা, স্বাস্থ্য, ভ্রমণ, প্রযুক্তি, খেলাধুলা).

- Adding a menu/routing system so ai choose the class for filtering.
Use metadata filtering to narrow down relevant answers (e.g., topic, difficulty).

- Implementing a basic RAG (Retrieval-Augmented Generation) to fetch answers from a FAQ dataset.

- Supporting Bangla input and output.

- Providing at least 3 FAQs per topic.

- Showing a fallback response if no answer is found.

- Keeping the interface simple (console or web).

**Demo video link:** https://drive.google.com/file/d/1esH30Cl80EC0Y_Rs6qH8HvHFda2DvjhN/view?usp=sharing

Installing Required Libraries

In [1]:
!pip install -qU langchain sentence-transformers transformers faiss-cpu langchain-community
!pip install -qU openai

print("Packages are installed uccessfully...")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.5 MB/s eta 0:00:00
Packages are installed uccessfully...


Importing Required Modules

In [2]:
import os
import faiss # vector Db
import numpy as np
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings # hugging face embedding model
from langchain_core.documents import Document # Data -> Document
from openai import OpenAI

print("Modules are installed successfully...")

Modules are installed successfully...


Setting-up Embeddings

In [3]:
embedding_model = HuggingFaceEmbeddings(model_name="l3cube-pune/bengali-sentence-similarity-sbert")

print("Embedding setup done..")

/tmp/ipython-input-1403789129.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="l3cube-pune/bengali-sentence-similarity-sbert")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding setup done..


Preparing Chunks with Metadata

In [4]:
education_chunks = [
    ("শিক্ষা মানুষের জ্ঞান ও দক্ষতা বৃদ্ধি করে।", {"category": "শিক্ষা"}),
    ("প্রাথমিক শিক্ষা শিশুর ভিত্তি গড়ে তোলে।", {"category": "শিক্ষা"}),
    ("শিক্ষায় প্রযুক্তির ব্যবহার শেখাকে সহজ করেছে।", {"category": "শিক্ষা"}),
    ("অনলাইন শিক্ষার মাধ্যমে ঘরে বসে পড়াশোনা করা যায়।", {"category": "শিক্ষা"}),
    ("শিক্ষা মানুষের চিন্তাশক্তি বিকাশে সাহায্য করে।", {"category": "শিক্ষা"}),
    ("উচ্চশিক্ষা কর্মজীবনে অগ্রগতির সুযোগ বাড়ায়।", {"category": "শিক্ষা"}),
    ("শিক্ষকরা শিক্ষার্থীদের ভবিষ্যৎ গঠনে গুরুত্বপূর্ণ ভূমিকা রাখেন।", {"category": "শিক্ষা"}),
    ("শিক্ষায় নিয়মিত অনুশীলন প্রয়োজন।", {"category": "শিক্ষা"}),
    ("কারিগরি শিক্ষা কর্মসংস্থানের সুযোগ তৈরি করে।", {"category": "শিক্ষা"}),
    ("শিক্ষা মানুষের নৈতিক মূল্যবোধ গড়ে তোলে।", {"category": "শিক্ষা"}),
    ("ডিজিটাল শিক্ষা ব্যবস্থায় স্মার্ট ক্লাসরুম জনপ্রিয় হচ্ছে।", {"category": "শিক্ষা"}),
    ("শিক্ষায় গবেষণা নতুন জ্ঞান সৃষ্টি করে।", {"category": "শিক্ষা"}),
    ("শিক্ষার্থীদের সৃজনশীলতা বিকাশে সহশিক্ষা কার্যক্রম প্রয়োজন।", {"category": "শিক্ষা"}),
    ("শিক্ষায় মূল্যায়ন পদ্ধতি গুরুত্বপূর্ণ।", {"category": "শিক্ষা"}),
    ("শিক্ষা দারিদ্র্য দূরীকরণে ভূমিকা রাখে।", {"category": "শিক্ষা"}),
    ("লাইব্রেরি শিক্ষা বিস্তারের একটি গুরুত্বপূর্ণ মাধ্যম।", {"category": "শিক্ষা"}),
    ("শিক্ষায় আত্মনির্ভরশীলতা তৈরি হয়।", {"category": "শিক্ষা"}),
    ("জীবনব্যাপী শিক্ষা মানুষকে আপডেট রাখে।", {"category": "শিক্ষা"}),
    ("শিক্ষায় সমান সুযোগ নিশ্চিত করা প্রয়োজন।", {"category": "শিক্ষা"}),
    ("শিক্ষা একটি জাতির মেরুদণ্ড।", {"category": "শিক্ষা"})
]


health_chunks = [
    ("স্বাস্থ্য ভালো থাকলে জীবন সুখকর হয়।", {"category": "স্বাস্থ্য"}),
    ("নিয়মিত ব্যায়াম স্বাস্থ্য ভালো রাখে।", {"category": "স্বাস্থ্য"}),
    ("পরিমিত খাবার গ্রহণ সুস্থ থাকার জন্য জরুরি।", {"category": "স্বাস্থ্য"}),
    ("স্বাস্থ্য সচেতনতা রোগ প্রতিরোধে সাহায্য করে।", {"category": "স্বাস্থ্য"}),
    ("পরিষ্কার পানি পান করা স্বাস্থ্যের জন্য গুরুত্বপূর্ণ।", {"category": "স্বাস্থ্য"}),
    ("নিয়মিত চিকিৎসা পরীক্ষা প্রয়োজন।", {"category": "স্বাস্থ্য"}),
    ("মানসিক স্বাস্থ্য শারীরিক স্বাস্থ্যের মতোই গুরুত্বপূর্ণ।", {"category": "স্বাস্থ্য"}),
    ("পর্যাপ্ত ঘুম স্বাস্থ্য ভালো রাখে।", {"category": "স্বাস্থ্য"}),
    ("ধূমপান স্বাস্থ্যের জন্য ক্ষতিকর।", {"category": "স্বাস্থ্য"}),
    ("স্বাস্থ্যকর খাবার রোগ প্রতিরোধ ক্ষমতা বাড়ায়।", {"category": "স্বাস্থ্য"}),
    ("স্বাস্থ্যবিধি মেনে চলা সংক্রমণ কমায়।", {"category": "স্বাস্থ্য"}),
    ("শিশুদের পুষ্টিকর খাবার দেওয়া প্রয়োজন।", {"category": "স্বাস্থ্য"}),
    ("স্বাস্থ্য সুরক্ষায় টিকা গুরুত্বপূর্ণ।", {"category": "স্বাস্থ্য"}),
    ("ডায়াবেটিস নিয়ন্ত্রণে খাদ্যাভ্যাস গুরুত্বপূর্ণ।", {"category": "স্বাস্থ্য"}),
    ("স্বাস্থ্য ঝুঁকি কমাতে নিয়মিত হাঁটা উপকারী।", {"category": "স্বাস্থ্য"}),
    ("মানসিক চাপ কমানো স্বাস্থ্যের জন্য ভালো।", {"category": "স্বাস্থ্য"}),
    ("স্বাস্থ্য শিক্ষা রোগ প্রতিরোধে সহায়ক।", {"category": "স্বাস্থ্য"}),
    ("অতিরিক্ত লবণ স্বাস্থ্যের জন্য ক্ষতিকর।", {"category": "স্বাস্থ্য"}),
    ("স্বাস্থ্যকর জীবনযাপন দীর্ঘায়ু নিশ্চিত করে।", {"category": "স্বাস্থ্য"}),
    ("স্বাস্থ্য সেবা মানুষের মৌলিক অধিকার।", {"category": "স্বাস্থ্য"})
]

travel_chunks = [
    ("ভ্রমণ মানসিক প্রশান্তি এনে দেয়।", {"category": "ভ্রমণ"}),
    ("ভ্রমণের মাধ্যমে নতুন সংস্কৃতি জানা যায়।", {"category": "ভ্রমণ"}),
    ("পরিকল্পিত ভ্রমণ খরচ কমায়।", {"category": "ভ্রমণ"}),
    ("ভ্রমণের সময় নিরাপত্তা গুরুত্বপূর্ণ।", {"category": "ভ্রমণ"}),
    ("ভ্রমণ মানুষকে নতুন অভিজ্ঞতা দেয়।", {"category": "ভ্রমণ"}),
    ("ভ্রমণে প্রয়োজনীয় কাগজপত্র সঙ্গে রাখা উচিত।", {"category": "ভ্রমণ"}),
    ("দেশ ভ্রমণে স্থানীয় খাবার উপভোগ করা যায়।", {"category": "ভ্রমণ"}),
    ("ভ্রমণ ক্লান্তি দূর করতে সাহায্য করে।", {"category": "ভ্রমণ"}),
    ("ভ্রমণের আগে আবহাওয়া জেনে নেওয়া দরকার।", {"category": "ভ্রমণ"}),
    ("পরিবারের সাথে ভ্রমণ আনন্দ বাড়ায়।", {"category": "ভ্রমণ"}),
    ("ভ্রমণে ছবি তোলা স্মৃতি ধরে রাখে।", {"category": "ভ্রমণ"}),
    ("ভ্রমণ মানুষকে উদার মানসিকতা শেখায়।", {"category": "ভ্রমণ"}),
    ("ভ্রমণে হালকা লাগেজ বহন করা ভালো।", {"category": "ভ্রমণ"}),
    ("পর্যটন শিল্প দেশের অর্থনীতিতে অবদান রাখে।", {"category": "ভ্রমণ"}),
    ("ভ্রমণে স্থানীয় নিয়মকানুন মানা উচিত।", {"category": "ভ্রমণ"}),
    ("ভ্রমণ নতুন বন্ধুত্ব তৈরি করে।", {"category": "ভ্রমণ"}),
    ("ভ্রমণে সময় ব্যবস্থাপনা জরুরি।", {"category": "ভ্রমণ"}),
    ("ভ্রমণের মাধ্যমে ইতিহাস জানা যায়।", {"category": "ভ্রমণ"}),
    ("ভ্রমণ মানসিক চাপ কমায়।", {"category": "ভ্রমণ"}),
    ("ভ্রমণ জীবনের একঘেয়েমি দূর করে।", {"category": "ভ্রমণ"})
]

technology_chunks = [
    ("প্রযুক্তি মানুষের জীবন সহজ করেছে।", {"category": "প্রযুক্তি"}),
    ("ইন্টারনেট প্রযুক্তির অন্যতম গুরুত্বপূর্ণ অংশ।", {"category": "প্রযুক্তি"}),
    ("কৃত্রিম বুদ্ধিমত্তা আধুনিক প্রযুক্তির বড় উদাহরণ।", {"category": "প্রযুক্তি"}),
    ("প্রযুক্তি শিক্ষা ব্যবস্থাকে আধুনিক করেছে।", {"category": "প্রযুক্তি"}),
    ("মোবাইল ফোন প্রযুক্তির বড় অবদান।", {"category": "প্রযুক্তি"}),
    ("প্রযুক্তির অপব্যবহার ক্ষতিকর হতে পারে।", {"category": "প্রযুক্তি"}),
    ("প্রযুক্তি ব্যবসার গতি বাড়িয়েছে।", {"category": "প্রযুক্তি"}),
    ("ডিজিটাল লেনদেন সময় বাঁচায়।", {"category": "প্রযুক্তি"}),
    ("প্রযুক্তি স্বাস্থ্যসেবাকে উন্নত করেছে।", {"category": "প্রযুক্তি"}),
    ("সাইবার নিরাপত্তা প্রযুক্তির গুরুত্বপূর্ণ অংশ।", {"category": "প্রযুক্তি"}),
    ("প্রযুক্তি কর্মসংস্থানের নতুন সুযোগ সৃষ্টি করেছে।", {"category": "প্রযুক্তি"}),
    ("রোবট প্রযুক্তি শিল্পক্ষেত্রে ব্যবহৃত হচ্ছে।", {"category": "প্রযুক্তি"}),
    ("প্রযুক্তি যোগাযোগ ব্যবস্থা সহজ করেছে।", {"category": "প্রযুক্তি"}),
    ("ডাটা বিশ্লেষণে প্রযুক্তি গুরুত্বপূর্ণ।", {"category": "প্রযুক্তি"}),
    ("প্রযুক্তির মাধ্যমে দূরশিক্ষা সম্ভব হয়েছে।", {"category": "প্রযুক্তি"}),
    ("প্রযুক্তি সময় ও শ্রম সাশ্রয় করে।", {"category": "প্রযুক্তি"}),
    ("প্রযুক্তি কৃষিতেও ব্যবহৃত হচ্ছে।", {"category": "প্রযুক্তি"}),
    ("প্রযুক্তি বিশ্বকে ছোট করেছে।", {"category": "প্রযুক্তি"}),
    ("প্রযুক্তি উদ্ভাবন উন্নয়নের চালিকা শক্তি।", {"category": "প্রযুক্তি"}),
    ("প্রযুক্তি ছাড়া আধুনিক জীবন কল্পনা করা কঠিন।", {"category": "প্রযুক্তি"})
]

sports_chunks = [
    ("খেলাধুলা শরীর সুস্থ রাখে।", {"category": "খেলাধুলা"}),
    ("নিয়মিত খেলাধুলা মানসিক স্বাস্থ্য ভালো রাখে।", {"category": "খেলাধুলা"}),
    ("খেলাধুলা দলগত কাজ শেখায়।", {"category": "খেলাধুলা"}),
    ("খেলাধুলা শিশুদের শারীরিক বিকাশে সহায়ক।", {"category": "খেলাধুলা"}),
    ("খেলাধুলা শৃঙ্খলা গড়ে তোলে।", {"category": "খেলাধুলা"}),
    ("ক্রিকেট বাংলাদেশে জনপ্রিয় খেলা।", {"category": "খেলাধুলা"}),
    ("ফুটবল বিশ্বব্যাপী জনপ্রিয়।", {"category": "খেলাধুলা"}),
    ("খেলাধুলা আত্মবিশ্বাস বাড়ায়।", {"category": "খেলাধুলা"}),
    ("খেলাধুলা মানসিক চাপ কমায়।", {"category": "খেলাধুলা"}),
    ("খেলাধুলা বিনোদনের ভালো মাধ্যম।", {"category": "খেলাধুলা"}),
    ("খেলাধুলা প্রতিযোগিতামূলক মনোভাব তৈরি করে।", {"category": "খেলাধুলা"}),
    ("খেলাধুলা শরীরের শক্তি বাড়ায়।", {"category": "খেলাধুলা"}),
    ("খেলাধুলা সামাজিক সম্পর্ক উন্নত করে।", {"category": "খেলাধুলা"}),
    ("অলিম্পিক খেলাধুলার বড় আসর।", {"category": "খেলাধুলা"}),
    ("খেলাধুলা সময়ানুবর্তিতা শেখায়।", {"category": "খেলাধুলা"}),
    ("খেলাধুলা তরুণদের মাদক থেকে দূরে রাখে।", {"category": "খেলাধুলা"}),
    ("খেলাধুলা নেতৃত্ব গুণ তৈরি করে।", {"category": "খেলাধুলা"}),
    ("খেলাধুলা মনোযোগ বৃদ্ধি করে।", {"category": "খেলাধুলা"}),
    ("খেলাধুলা সুস্থ জীবনের অংশ।", {"category": "খেলাধুলা"}),
    ("খেলাধুলা জীবনে আনন্দ আনে।", {"category": "খেলাধুলা"})
]


In [5]:
all_chunks = education_chunks + health_chunks + travel_chunks + technology_chunks + sports_chunks

Creating Vector Store with Metadata

In [6]:
documents = [Document(page_content=text, metadata=meta) for text, meta in all_chunks]

documents[:5]

[Document(metadata={'category': 'শিক্ষা'}, page_content='শিক্ষা মানুষের জ্ঞান ও দক্ষতা বৃদ্ধি করে।'),
 Document(metadata={'category': 'শিক্ষা'}, page_content='প্রাথমিক শিক্ষা শিশুর ভিত্তি গড়ে তোলে।'),
 Document(metadata={'category': 'শিক্ষা'}, page_content='শিক্ষায় প্রযুক্তির ব্যবহার শেখাকে সহজ করেছে।'),
 Document(metadata={'category': 'শিক্ষা'}, page_content='অনলাইন শিক্ষার মাধ্যমে ঘরে বসে পড়াশোনা করা যায়।'),
 Document(metadata={'category': 'শিক্ষা'}, page_content='শিক্ষা মানুষের চিন্তাশক্তি বিকাশে সাহায্য করে।')]

In [7]:
# vector_store = FAISS.from_documents(documents, embedding_model)

Defining Metadata Filter Function

In [8]:
def filter_by_metadata(query, category):
    """
    Filter vector store by category metadata first, then perform similarity search.
    """
    print("\n[Metadata Filtering] Category:", category)
    filtered_docs = [doc for doc in documents if doc.metadata['category'] == category] ## category
    # for doc in filtered_docs:
    #     print(" -", doc.page_content)

    if not filtered_docs:
        return []
    temp_vector_store = FAISS.from_documents(filtered_docs, embedding_model)
    similar_docs = temp_vector_store.similarity_search(query, k=3)

    print("\n🕵🏻 [Similarity Search Results for Query]", query)
    for doc in similar_docs:
        print(" >>", doc.page_content)

    return similar_docs

Settingup GitHub-hosted OpenAI LLM (via OpenAI SDK)

In [9]:
from google.colab import userdata
userdata.get('git_api_key1')

os.environ['GITHUB_TOKEN'] = userdata.get('git_api_key1')  # Replace with actual token
token = os.environ['GITHUB_TOKEN']
endpoint = "https://models.github.ai/inference"
model = "openai/gpt-4.1-nano"

client = OpenAI(
    base_url=endpoint,
    api_key=token,
)

Defining Final RAG Chain

In [10]:
def ask_faq_bot(user_question: str, category: str):
    print("\n[User Question]", user_question)
    docs = filter_by_metadata(user_question, category)
    context = "\n".join([doc.page_content for doc in docs])
    response = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "তুমি একজন সহায়ক বাংলা সহকারী। শুধুমাত্র নিচের প্রাসঙ্গিক তথ্য থেকে উত্তর দাও। যদি প্রশ্নের উত্তর এতে না থাকে, বলো 'দুঃখিত, এই বিষয়ে আমার উত্তরটি জানা নেই। যদি প্রসঙ্গ উপস্থিত না থাকে তবে অবশ্যই প্রশ্নের উত্তর দেবেন না, উত্তর দাও 'আমার এই প্রসঙ্গ সম্পর্কিত কিছু জানা নেই।' " + context,
            },
            {
                "role": "user",
                "content": user_question,
            }
        ],
        temperature=0.7,
        top_p=0.7,
        model=model
    )
    return response.choices[0].message.content

In [11]:
def detect_category_llm(question):
    """
    Use LLM to determine appropriate category
    """
    system_msg = "তুমি একটি শ্রেণিবিন্যাসকারী এজেন্ট। নিচের প্রশ্নটি পড়ে বলো এটি কোন ক্যাটাগরির মধ্যে পড়ে: শিক্ষা, স্বাস্থ্য, ভ্রমণ, প্রযুক্তি, খেলাধুলা । শুধুমাত্র ক্যাটাগরির নাম এক শব্দে বাংলায় উত্তর দাও। যদি প্রশ্নটি কোনও ক্যাটাগরির না হয়, তাহলে ভ্রমণ ক্যাটাগরি নির্বাচন করো।"
    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": question}
        ],
        model=model,
        temperature=0,
        top_p=1.0
    )
    category = response.choices[0].message.content.strip().lower()
    print("[LLM Router] Category:", category)
    return category

Interacting with FAQ Bot

In [12]:
#Quick-test

# question = input("আপনার প্রশ্নটি বলুন: ")
# category = detect_category_llm(question)
# response = ask_faq_bot(question, category)
# print("প্রত্যাশিত উত্তর:", response)

In [16]:
# Interactive loop
print("********************************************************************************************\n\n")
print("🙋🏻‍♂️ বাংলা FAQ সহকারীর সাথে চ্যাট করুন! 🚩প্রস্থান করতে 'quit', 'exit' অথবা 'bye' টাইপ করুন।\n\n")
print("********************************************************************************************\n")
while True:
    user_input = input("👩🏻‍💻 আপনার প্রশ্নটি বলুন: ")
    if user_input.lower() in ["quit", "exit", "bye"]:
        print("\nআপনি প্রস্তুত হলে আবার আসবেন! 🙋🏻‍♂️\n")
        break

    # First try exact match
    category = detect_category_llm(user_input)
    answer = ask_faq_bot(user_input, category)

    print("\n💡প্রত্যাশিত উত্তর:", answer)
    print("\n")

********************************************************************************************


🙋🏻‍♂️ বাংলা FAQ সহকারীর সাথে চ্যাট করুন! 🚩প্রস্থান করতে 'quit', 'exit' অথবা 'bye' টাইপ করুন।


********************************************************************************************

👩🏻‍💻 আপনার প্রশ্নটি বলুন: পরিবারের সাথে ভ্রমণের সুবিধা কী?
[LLM Router] Category: ভ্রমণ

[User Question] পরিবারের সাথে ভ্রমণের সুবিধা কী?

[Metadata Filtering] Category: ভ্রমণ

🕵🏻 [Similarity Search Results for Query] পরিবারের সাথে ভ্রমণের সুবিধা কী?
 >> পরিবারের সাথে ভ্রমণ আনন্দ বাড়ায়।
 >> ভ্রমণ মানুষকে নতুন অভিজ্ঞতা দেয়।
 >> ভ্রমণ জীবনের একঘেয়েমি দূর করে।

💡প্রত্যাশিত উত্তর: পরিবারের সাথে ভ্রমণের সুবিধা হলো এটি আনন্দ বাড়ায়, নতুন অভিজ্ঞতা দেয় এবং জীবনের একঘেয়েমি দূর করে।


👩🏻‍💻 আপনার প্রশ্নটি বলুন: নিয়মিত ব্যায়াম স্বাস্থ্যের ওপর কী প্রভাব ফেলে?
[LLM Router] Category: স্বাস্থ্য

[User Question] নিয়মিত ব্যায়াম স্বাস্থ্যের ওপর কী প্রভাব ফেলে?

[Metadata Filtering] Category: স্বাস্থ্য

🕵🏻 [Similarity Search Re